# Temperature dependent elastic constants

## Background

$$C_{ijkl} = \frac{1}{V} \frac{\partial^2 U}{\partial \varepsilon_{ij}\partial \varepsilon_{kl}}$$

$$U(T) = \frac{V}{2}C_{ijkl}(T)\varepsilon_{ij}\varepsilon_{kl}$$

$$\sigma_{ij} = C_{ijkl}{\varepsilon_{kl}}$$

### How to get $U$ or $\sigma$

- MD
- Quasi-Harmonic

## Tasks

- Get $a_0$ from potential
- Lattice parameter (as a function of T)
  - MD
    - NVT
    - NPT
  - QH
- Calculate $U$ or $\sigma$ for various $\varepsilon$
  - MD: Equilibriate and average with LAMMPS
  - QH: Get strains from Yuriy's tool and run phonopy
- Fit

## Teams

- MD: Erik, Han, (Raynol), Prabhath, Jan
- QH: Raynol, (Sam), Bharathi, Ahmed, Haitham
- Fit & Yuriy: Sam
- Literature

# Implementation

* https://atomistics.readthedocs.io/en/latest/bulk_modulus_with_gpaw.html#elastic-matrix
* https://github.com/pyiron/atomistics/blob/main/tests/test_elastic_lammpslib_functional.py
* https://github.com/pyiron/pyiron_workflow_atomistics/blob/interstitials/pyiron_workflow_atomistics/dataclass_storage.py
* https://github.com/ligerzero-ai/pyiron_workflow_lammps/blob/main/pyiron_workflow_lammps/engine.py#L21

## Imports

In [1]:
from ase.build import bulk
from ase.atoms import Atoms

In [2]:
import numpy as np
import pandas as pd

from atomistics.workflows.elastic.workflow import (
    analyse_structures_helper,
    generate_structures_helper,
)

from atomistics.calculators import evaluate_with_lammpslib, get_potential_by_name
from atomistics.calculators.lammps.libcalculator import calc_static_with_lammpslib
from atomistics.calculators import calc_molecular_dynamics_npt_with_lammpslib
from atomistics.calculators import calc_molecular_dynamics_nvt_with_lammpslib

## Create bulk sample

In [3]:
structure = bulk('Al', 'fcc', a=4.05, cubic=True)

In [4]:
potential_name = "1999--Mishin-Y--Al--LAMMPS--ipr1"

In [5]:
df_pot_selected = get_potential_by_name(
    potential_name=potential_name
)

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


## 0K Relaxed Structure

In [6]:
def get_relaxed_structure(structure: Atoms, potential: str) -> Atoms:
    
    df_pot_selected = get_potential_by_name(
            potential_name=potential
        )
    
    result_dict = evaluate_with_lammpslib(
            task_dict={"optimize_positions_and_volume": structure},
            potential_dataframe=df_pot_selected,
        )
    
    structure_relaxed = result_dict['structure_with_optimized_positions_and_volume']

    return structure_relaxed

In [7]:
relaxed_structure = get_relaxed_structure(structure, potential_name)
relaxed_structure

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
--------------------------------------------------------------------------

  Local host:   cmti001
  Local device: hfi1_0
--------------------------------------------------------------------------


Atoms(symbols='Al4', pbc=True, cell=[4.050004662201837, 4.050004662201837, 4.050004662201837])

## 0K Lattice Constant

In [8]:
def get_minimum_lattice_constant(structure: Atoms, potential: str) -> float:

    structure_relaxed = get_relaxed_structure(structure, potential)
    a_0 = structure_relaxed.get_volume()**(1/3) #Angstrom

    return a_0

In [9]:
a_0 = get_minimum_lattice_constant(structure, potential_name)
a_0

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


4.050004662201837

## 0K Elastic Constants

In [10]:
def get_elastic_constant_after_deformation(structure : Atoms, potential_dataframe : pd.DataFrame, elastic_constant : str, deformation_gradient : np.array, strain : float) -> float:

    structure_strained = structure.copy()
    relaxed_cell = np.array(structure_strained.get_cell().tolist())

    strained_cell = deformation_gradient@relaxed_cell
    structure_strained.set_cell(strained_cell, scale_atoms=True)

    relaxed_dict = calc_static_with_lammpslib(structure=structure, potential_dataframe=potential_dataframe)
    strained_dict = calc_static_with_lammpslib(structure=structure_strained, potential_dataframe=potential_dataframe)

    # print("Relaxed stress:\n", relaxed_dict['stress'])
    # print("Strained stress:\n", strained_dict['stress'])

    diff = strained_dict['stress'] - relaxed_dict['stress']

    if elastic_constant == 'C11':
        constant = diff[0, 0] / strain

    elif elastic_constant == 'C12':
        sigma33 = diff[2, 2]
        constant = (sigma33/ strain) / 2

    elif elastic_constant == 'C44':
        sigma23 = diff[2, 1]
        constant = sigma23 / (2 * strain)

    return abs(constant)


In [11]:
def calculate_elastic_constants(structure : Atoms, potential : str, strain : float = 0.005) -> list:

    df_pot_selected = get_potential_by_name(
        potential_name=potential
    )

    elastic_constants_list = []

    deformation_gradient_dict = {
        'C11': np.eye(3,3) + np.array([[ strain,      0,      0], 
                                       [      0,      0,      0],
                                       [      0,      0,      0]]),
        'C12': np.eye(3,3) + np.array([[ strain,      0,      0], 
                                       [      0, strain,      0], 
                                       [      0,      0,      0]]),
        'C44': np.eye(3,3) + np.array([[      0,      0,      0], 
                                       [      0,      0, strain], 
                                       [      0, strain,      0]])
    }

    for constant_str, deformation_gradient in deformation_gradient_dict.items():
        const = get_elastic_constant_after_deformation(structure=structure, 
                                                       potential_dataframe=df_pot_selected, 
                                                       elastic_constant=constant_str, 
                                                       deformation_gradient=deformation_gradient,
                                                       strain=strain)
        elastic_constants_list.append(const)

    return elastic_constants_list

In [12]:
calculate_elastic_constants(structure=relaxed_structure, potential=potential_name, strain=-0.005)

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/helpers.py:29: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  lmp.interactive_structure_setter(


[1145843.555110171, 613579.6671739953, 316998.27964768995]

## Finite Temperature equlibiration
* First run NPT to relax volume
* Then equilibriate the cell by running NVT

In [13]:
# FIXME: Make it for a generic element
def equilibriate_structure_at_finite_temperature(structure : Atoms, potential : str, temperature : float = 500) -> Atoms:

    df_pot_selected = get_potential_by_name(
        potential_name=potential
    )
    structure_repeated = structure.repeat(5)

    npt_dict = calc_molecular_dynamics_npt_with_lammpslib(
        structure=structure_repeated,
        potential_dataframe=df_pot_selected,
        Tstart=temperature,
        Tstop=temperature,
        run=10000,
        thermo=100
    )
    npt_lattice_constant = (np.mean(npt_dict['volume'][20:]/len(structure_repeated))*len(structure))**(1/3)
    
    # structure_repeated_old = bulk('Al', a=npt_lattice_constant, cubic=True).repeat(5)
    structure_repeated_npt = structure.copy()
    structure_repeated_npt.set_cell(
        [[npt_lattice_constant,0,0], [0,npt_lattice_constant,0], [0,0,npt_lattice_constant]],
        scale_atoms = True
    )
    structure_repeated_npt = structure_repeated_npt.repeat(5)

    nvt_dict = calc_molecular_dynamics_nvt_with_lammpslib(
        structure=structure_repeated_npt,
        potential_dataframe=df_pot_selected,
        Tstart=temperature,
        Tstop=temperature,
        run=10000,
        thermo=100
    )    
    structure_repeated_nvt = structure_repeated_npt.copy()
    structure_repeated_nvt.set_cell(
        nvt_dict['cell'][-1]
    )
    structure_repeated_nvt.set_positions(
        nvt_dict['positions'][-1]
    )
    structure_repeated_nvt.set_velocities(
        nvt_dict['velocities'][-1]
    )

    return structure_repeated_nvt

In [14]:
structure_repeated_nvt_300 = equilibriate_structure_at_finite_temperature(structure=relaxed_structure, potential=potential_name, temperature=300)
structure_repeated_nvt_500 = equilibriate_structure_at_finite_temperature(structure=relaxed_structure, potential=potential_name, temperature=500)
structure_repeated_nvt_800 = equilibriate_structure_at_finite_temperature(structure=relaxed_structure, potential=potential_name, temperature=800)

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopy

In [19]:
structure_repeated_nvt_300, structure_repeated_nvt_500, structure_repeated_nvt_800

(Atoms(symbols='Al500', pbc=True, cell=[20.324519140674408, 20.324519140674408, 20.324519140674408], momenta=...),
 Atoms(symbols='Al500', pbc=True, cell=[20.390245415624175, 20.390245415624175, 20.390245415624175], momenta=...),
 Atoms(symbols='Al500', pbc=True, cell=[20.51366823684186, 20.51366823684186, 20.51366823684186], momenta=...))

## Finite Temperature Elastic Constants

In [15]:
def get_elastic_constant_after_deformation_at_finite_temperature(structure : Atoms, potential_dataframe : pd.DataFrame, elastic_constant : str, deformation_gradient : np.array, temperature : float, strain : float):

    from atomistics.calculators.lammps.libcalculator import calc_molecular_dynamics_nvt_with_lammpslib # FIXME: Add to top later!

    structure_strained = structure.copy()
    relaxed_cell = np.array(structure_strained.get_cell().tolist())

    strained_cell = deformation_gradient@relaxed_cell
    structure_strained.set_cell(strained_cell, scale_atoms=True)

    relaxed_dict = calc_molecular_dynamics_nvt_with_lammpslib(structure=structure, 
                                                              potential_dataframe=potential_dataframe,
                                                              Tstart=temperature,
                                                              Tstop=temperature,
                                                              run=10000,
                                                              thermo=100)
    strained_dict = calc_molecular_dynamics_nvt_with_lammpslib(structure=structure_strained, 
                                                              potential_dataframe=potential_dataframe,
                                                              Tstart=temperature,
                                                              Tstop=temperature,
                                                              run=10000,
                                                              thermo=100)

    # print("Relaxed stress:\n", relaxed_dict['pressure'])
    # print("Strained stress:\n", strained_dict['pressure'])

    diff = -np.mean(strained_dict['pressure'] - relaxed_dict['pressure'], axis=0)
    # print(diff)

    if elastic_constant == 'C11':
        constant = diff[0, 0] / strain

    elif elastic_constant == 'C12':
        sigma33 = diff[2, 2]
        constant = (sigma33/ strain) / 2

    elif elastic_constant == 'C44':
        sigma23 = diff[2, 1]
        constant = sigma23 / (2 * strain)

    return abs(constant)


In [16]:
def calculate_elastic_constants_at_finite_temperature(structure : Atoms, potential : str, temperature : float = 500, strain : float = 0.005) -> list:

    df_pot_selected = get_potential_by_name(
        potential_name=potential
    )

    elastic_constants_list = []

    deformation_gradient_dict = {
        'C11': np.eye(3,3) + np.array([[ strain,      0,      0], 
                                       [      0,      0,      0],
                                       [      0,      0,      0]]),
        'C12': np.eye(3,3) + np.array([[ strain,      0,      0], 
                                       [      0, strain,      0], 
                                       [      0,      0,      0]]),
        'C44': np.eye(3,3) + np.array([[      0,      0,      0], 
                                       [      0,      0, strain], 
                                       [      0, strain,      0]])
    }

    for constant_str, deformation_gradient in deformation_gradient_dict.items():
        const = get_elastic_constant_after_deformation_at_finite_temperature(structure=structure, 
                                                                                potential_dataframe=df_pot_selected, 
                                                                                elastic_constant=constant_str, 
                                                                                deformation_gradient=deformation_gradient,
                                                                                temperature=temperature,
                                                                                strain=strain)
        elastic_constants_list.append(const)

    return elastic_constants_list

In [17]:
elastic_constants_list_300 = calculate_elastic_constants_at_finite_temperature(structure=structure_repeated_nvt_300, potential=potential_name, temperature=300, strain=0.005)
elastic_constants_list_500 = calculate_elastic_constants_at_finite_temperature(structure=structure_repeated_nvt_500, potential=potential_name, temperature=500, strain=0.005)
elastic_constants_list_800 = calculate_elastic_constants_at_finite_temperature(structure=structure_repeated_nvt_800, potential=potential_name, temperature=800, strain=0.005)

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/helpers.py:29: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  lmp.interactive_structure_setter(
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [18]:
elastic_constants_list_300, elastic_constants_list_500, elastic_constants_list_800

([1100747.2692788003, 629290.6606284853, 333452.3055876522],
 [1051359.6491810333, 614936.9099964361, 314049.15498242853],
 [822006.9916656188, 511164.97334666474, 250652.86079160072])

## (Jan + Yury)'s function to fit elastic constants

In [29]:
sym_dict, structure_dict = generate_structures_helper(
    structure=relaxed_structure,
    eps_range=0.005,
    num_of_point=5,
    zero_strain_job_name="s_e_0",
    sqrt_eta=True,
)
structure_dict

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/workflows/elastic/symmetry.py:32: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  SGN = dataset["number"]


OrderedDict([('s_e_0',
              Atoms(symbols='Al4', pbc=True, cell=[4.050004662201837, 4.050004662201837, 4.050004662201837])),
             ('s_01_e_m0_00500',
              Atoms(symbols='Al4', pbc=True, cell=[4.0297037591141, 4.0297037591141, 4.0297037591141])),
             ('s_01_e_m0_00250',
              Atoms(symbols='Al4', pbc=True, cell=[4.039866962542076, 4.039866962542076, 4.039866962542076])),
             ('s_01_e_0_00250',
              Atoms(symbols='Al4', pbc=True, cell=[4.060117049134704, 4.060117049134704, 4.060117049134704])),
             ('s_01_e_0_00500',
              Atoms(symbols='Al4', pbc=True, cell=[4.07020431200885, 4.07020431200885, 4.07020431200885])),
             ('s_08_e_m0_00500',
              Atoms(symbols='Al4', pbc=True, cell=[4.0297037591141, 4.0297037591141, 4.050004662201837])),
             ('s_08_e_m0_00250',
              Atoms(symbols='Al4', pbc=True, cell=[4.039866962542076, 4.039866962542076, 4.050004662201837])),
             ('s_

In [32]:
df_pot_selected = get_potential_by_name(
    potential_name=potential_name
)

result_dict = evaluate_with_lammpslib(
    task_dict={"calc_energy": structure_dict},
    potential_dataframe=df_pot_selected,
)

sym_dict, elastic_dict = analyse_structures_helper(
    output_dict=result_dict,
    sym_dict=sym_dict,
    fit_order=2,
    zero_strain_job_name="s_e_0",
)

sym_dict, elastic_dict

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/helpers.py:29: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  lmp.interactive_structure_setter(


({'SGN': 225,
  'v0': 66.43035441556098,
  'LC': 'CI',
  'Lag_strain_list': ['01', '08', '23'],
  'epss': array([-0.005 , -0.0025,  0.    ,  0.0025,  0.005 ]),
  'strain_energy': [[(-0.005, -13.436320248980278),
    (-0.0025, -13.439079680886989),
    (0.0, -13.439999952735112),
    (0.0024999999999999996, -13.439084974614394),
    (0.005, -13.436364320399795)],
   [(-0.005, -13.43817471490433),
    (-0.0025, -13.439544638502635),
    (0.0, -13.439999952735112),
    (0.0024999999999999996, -13.43954822781134),
    (0.005, -13.43820419261515)],
   [(-0.005, -13.437971451918393),
    (-0.0025, -13.439501038418326),
    (0.0, -13.439999952735112),
    (0.0024999999999999996, -13.439515785430654),
    (0.005, -13.438089441277945)]],
  'e0': -13.439999952735112,
  'A2': array([2.20130388, 1.08985578, 1.1861949 ])},
 {'elastic_matrix': array([[114.103117  ,  60.51102935,  60.51102935,   0.        ,
            0.        ,   0.        ],
         [ 60.51102935, 114.103117  ,  60.51102935,   0

In [52]:
def fit_elastic_constants(structure: Atoms, potential: str, strains, stresses=None, energies=None):

    sym_dict, structure_dict = generate_structures_helper(
        structure=structure,
        eps_range=0.005,
        num_of_point=5,
        zero_strain_job_name="s_e_0",
        sqrt_eta=True,
    )

    df_pot_selected = get_potential_by_name(
        potential_name=potential
    )

    result_dict = evaluate_with_lammpslib(
        task_dict={"calc_energy": structure_dict},
        potential_dataframe=df_pot_selected,
    )

    sym_dict, elastic_dict = analyse_structures_helper(
        output_dict=result_dict,
        sym_dict=sym_dict,
        fit_order=2,
        zero_strain_job_name="s_e_0",
    )

    return elastic_dict

In [53]:
relaxed_structure = get_relaxed_structure(structure, potential_name)
elast_dict = fit_elastic_constants(
    structure=relaxed_structure,
    potential=potential_name,
    strains=None)
elast_dict['elastic_matrix']

/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/workflows/elastic/symmetry.py:32: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  SGN = dataset["number"]
/cmmc/ptmp/pyironhb/pyiron_latest_env/lib/python3.11/site-packages/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

array([[114.103117  ,  60.51102935,  60.51102935,   0.        ,
          0.        ,   0.        ],
       [ 60.51102935, 114.103117  ,  60.51102935,   0.        ,
          0.        ,   0.        ],
       [ 60.51102935,  60.51102935, 114.103117  ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  31.67489592,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         31.67489592,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  31.67489592]])

In [ ]:
def get_bulk_structure(
    name: str,
    crystalstructure=None,
    a=None,
    b=None,
    c=None,
    alpha=None,
    covera=None,
    u=None,
    orthorhombic=False,
    cubic=False,
    basis=None,
):
    from ase.build import bulk
    equil_struct = bulk(
        name=name,
        crystalstructure=crystalstructure,
        a=a,
        b=b,
        c=c,
        alpha=alpha,
        covera=covera,
        u=u,
        orthorhombic=orthorhombic,
        cubic=cubic,
        basis=basis,
    )
    return equil_struct